---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [14]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [15]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [16]:
def graph_identification():
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [17]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [11]:
def salary_predictions():
    import math
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
#     from sklearn.model_selection import train_test_split
#     from sklearn.metrics import roc_auc_score

    df = pd.DataFrame(index = G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['dc'] = pd.Series(nx.degree_centrality(G))
    df['cc'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['bc'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))
    df['pr'] = pd.Series(nx.pagerank(G))
    df['hub'] = pd.Series(nx.hits(G)[0])

    mask = df['ManagementSalary'].apply(lambda x: math.isnan(x))
    df_pred = df[mask]
    df = df[~mask]

    df_X_pred = df_pred[['Department', 'clustering', 'degree', 'dc', 'cc', 'bc', 'pr', 'hub']]
    X = df[['Department', 'clustering', 'degree', 'dc', 'cc', 'bc', 'pr', 'hub']]
    y = df['ManagementSalary']

#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaler = MinMaxScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
    X = scaler.fit_transform(X)
    X_pred = scaler.transform(df_X_pred)
    
    clf = MLPClassifier(hidden_layer_sizes = [10, 10], 
                        activation ='relu', alpha = 1, solver='lbfgs').fit(X, y)
#     y_pred = clf.predict(X_test)
#     score = roc_auc_score(y_test, y_pred)
    
    test_proba = clf.predict_proba(X_pred)[:, 1]
    return pd.Series(test_proba,df_X_pred.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [18]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [87]:
def new_connections_predictions():
    import math
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['pa'] = [x[2] for x in preferential_attachment]
    df['cn'] = [len(list(nx.common_neighbors(G, u, v))) for u, v in df.index]
    df['jc'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df['rai'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['aai'] = [x[2] for x in list(nx.adamic_adar_index(G))]
    df['cn_sh'] = [x[2] for x in list(nx.cn_soundarajan_hopcroft(G, community='Department'))]
    df['rai_sh'] = [x[2] for x in list(nx.ra_index_soundarajan_hopcroft(G, community='Department'))]
    df = pd.concat([df, future_connections], axis=1, join='inner')
    
    mask = df['Future Connection'].apply(lambda x: math.isnan(x))
    df_pred = df[mask]
    df = df[~mask]
    
    X = df[['pa', 'cn', 'jc', 'rai', 'aai', 'cn_sh', 'rai_sh']]
    y = df['Future Connection']
    df_X_pred = df_pred[['pa', 'cn', 'jc', 'rai', 'aai', 'cn_sh', 'rai_sh']]

#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    scaler = MinMaxScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
    X = scaler.fit_transform(X)
    X_pred = scaler.transform(df_X_pred)
    
    clf = MLPClassifier(hidden_layer_sizes = [10, 10], 
                        activation ='relu', alpha = 1, solver='lbfgs').fit(X, y)
#     y_pred = clf.predict(X_test)
#     score = roc_auc_score(y_test, y_pred)
    
    test_proba = clf.predict_proba(X_pred)[:, 1]
    return pd.Series(test_proba,df_X_pred.index)